# Parallel and Distributed Low Depth Algorithms for QAE

## Summary

This notebook shows how to practically run **Monte Carlo integration** using a parallelized and distributed version of the **Power Law Amplitude Estimation** (PLAE) algorithm for quantum amplitude estimation. The PLAE algorithm is a low depth algorithm utilizing maximum likelihood estimation based on the measurements produced by different quantum circuits.

PLAE paper: https://arxiv.org/abs/2012.03348

## Monte Carlo integration as QAE problem

Monte Carlo integration can be used to calculate the expected value of a real-valued function $0\leq f(x)\leq 1$ defined for $n$-bit input $x\in \{0,1\}^n$ with probability $p(x)$:
$$\mathbb{E}\big{[}f(x)\big{]}=\sum^{2^n-1}_{x=0}p(x)f(x).$$
We can define the unitary operators $\mathscr{R}$ and $\mathscr{P}$ as
$$\mathscr{R} |x_n \rangle |0\rangle = |x\rangle _n \Big{(}\sqrt{f(x)} |1\rangle + \sqrt{1-f(x)}|0\rangle \Big{)}$$
$$\mathscr{P} |0\rangle _n = \sum^{2^n-1}_{x=0} \sqrt{p(x)}|x\rangle _n, $$
and apply them to $|0\rangle _{n+1}$ to generate the state $|\Psi \rangle$:
$$|\Psi \rangle =\mathscr{R}\big{(}\mathscr{P}\otimes I_1 \big{)}|0\rangle _n |0 \rangle = \sum^{2^n-1}_{x=0} \sqrt{p(x)}|x\rangle _n \Big{(}\sqrt{f(x)}|1\rangle + \sqrt{1-f(x)}|0\rangle \Big{)}, $$
where $I_1$ is the identity acting on a qubit. We can define $a:=\mathbb{E}\big{[}f(x)\big{]}$ and introduce two orthonormal bases:
$$|\tilde{\Psi}_1 \rangle = \dfrac{1}{\sqrt{a}}\sum^{2^n-1}_{x=0}\sqrt{p(x)}\sqrt{f(x)}|x \rangle _n |1 \rangle ,$$
$$|\tilde{\Psi}_0 \rangle = \dfrac{1}{\sqrt{1-a}}\sum^{2^n-1}_{x=0}\sqrt{p(x)}\sqrt{1-f(x)}|x \rangle _n |0 \rangle .$$

The state $|\Psi \rangle $ can now be rewritten as
$$|\Psi \rangle = \sqrt{a}|\tilde{\Psi} _1 \rangle+\sqrt{1-a}|\tilde{\Psi} _0 \rangle. $$
The Monte Carlo integration problem can be therefore reduced to an amplitude estimation of $\tilde{\Psi} _1$.

## $\mathscr{R}$ and $\mathscr{P}$ operators

The forms of the $\mathscr{R}$ and $\mathscr{P}$ operators are specific to the problem we are trying to solve. The generic `UnitaryOperator` class in `qae/operators.py` lays out the basic attributes and methods every operator should provide. 

## Sine Squared Integral

As an example application of Monte Carlo integration, we want to calculate the integral:
$$\dfrac{1}{\pi/5}\int ^{\frac{\pi}{5}}_0 \sin ^2 (x) \, dx. $$

The operator $\mathscr{R}$ can be constructed with controlled-$Y$ rotations, $\mathscr{P}$ with Hadamard gates. The `SineSquaredOperator` class in `qae/applications/montecarlo/montecarlo_operators.py`, implementing the actual $\mathscr{R}$ and $\mathscr{P}$ operators for this problem, inherits from `UnitaryOperator`. Any different problem should implement its own `ProblemOperator` class derived from `UnitaryOperator`.

## Parallelization

Given the nature of the PLAE algorithm, we can parallelize the computation of each likelihood function, eventually merging them into a single likelihood function, on which the estimation will be performed. We assume we have multiple quantum processors with different numbers of qubits: such parameters are defined in `parallelization/hardware_configuration.py`.

### Parallelization - Scheduling

Our goal consists into writing down a distribution schedule optimizing the allocation of qubits needed by the algorithm given the available resources. One can then simply follow the schedule and run over the distributed system to obtain an estimate of the expectation value. The `Schedule` class in `parallelization/scheduling/schedule.py` represents a generic scheduling, in which the method `make_schedule` is abstract: this method will be implemented by the children and will define the specific schedule.

The `GreedySchedule` class inherits from `Schedule` and greedily fills up the QPUs with as many states that can possibly fit; the remaining needed qubits are then split across QPUs; when the QPUs cannot fit any more states, the execution of those estimations are moved to the next round.

As a practical example, let's assume we have 3 QPUs with 10, 10 and 5 qubits respectively, and let's assume that our algorithm requires 5 qubits to run (including ancillas) and must run the circuit 7 times. A greedy schedule can be written as follows:

```
Round 1:
[(0, [5, 0, 0]), (1, [0, 5, 0]), (2, [0, 5, 0]), (3, [5, 0, 0]), (4, [0, 0, 5])]
Round 2:
[(0, [5, 0, 0]), (1, [0, 5, 0])]
```

In the first round, we allocate 5 qubits in QPU1 (first circuit), 5 qubits in QPU2 (second circuit), 5 qubits in QPU2 (third), 5 qubits in QPU1 (fourth) and 5 qubits in QPU3 (fifth). We still need to run the circuit twice, however we have fully allocated all the qubits available in the QPUs. We therefore move the execution of the sixth and the seventh circuit to a second round.

Notice that other approaches are possible for resources allocation, for instance constraint programming. One can implement his/her own scheduling algorithm by inheriting from the `Schedule` class.

### Parallelization - Algorithm

The class `ParallelMaximumLikelihoodEstimation` (PMLE) in `parallelization/algorithms/amplitude_estimators/parallel_mlae.py` is built on top of the Qiskit built-in class `MaximumLikelihoodEstimation` (MLE). Unlike the MLE class, PMLE requires a distribution schedule to be instantiated, as well as a `ParallelQuantumInstance` class, defined in `parallelization/utils/parallel_quantum_instance.py`, which inherits from the Qiskit built-in `QuantumInstance` class.

The PMLE class runs in parallel the circuits, computes the maximum likelihood functions, and eventually joins them in order to obtain the final estimate.

## Run the program

The main program is `parallel_montecarlo.py`. One can run it simply with `./parallel_montecarlo.py`.

### Run the program - Output

This is an example output of the program:

```

### Configuration ###
# Algorithm Configuration #
N qubits: 3, N circuits: 7
# Hardware Configuration #
N QPUs: 3, qubits per qpu: [10, 10, 5]
# Integral Configuration #
Problem: sine_squared, b_max: 0.6283185307179586

### Schedule for parallelization ###
# Round 1 #
[(0, [5, 0, 0]), (1, [0, 5, 0]), (2, [0, 5, 0]), (3, [5, 0, 0]), (4, [0, 0, 5])]
# Round 2 #
[(0, [5, 0, 0]), (1, [0, 5, 0])]

### Results ###
Analytical result: 0.12158663567967151
Discretized result: 0.1211973148745352
Estimation result: 0.12084395622049497
```

In the first part, the program prints out the configuration. In the second part, one can see the schedule for parallelization. In the third part, the results are printed out: the analytical result (when available, like in our case), the discretized result (obtained from the discretization of the integral) and the estimation result, obtained by running the MLAE algorithm.

### Run the program - Code

Let's now run the main program in `parallel_montecarlo.py`. Let's start by importing the needed packages:

In [4]:
import math

from qae.applications.montecarlo.montecarlo_operators import SineSquaredOperator
from qae.applications.montecarlo.montecarlo_config import QAEMonteCarloConfig, IntegralConfig

from qiskit import Aer
from qiskit.algorithms.amplitude_estimators import EstimationProblem

from qae import UnitaryOperator
from parallelization import ParallelMaximumLikelihoodAmplitudeEstimation
from parallelization import ParallelQuantumInstance
from parallelization import GreedySchedule

from typing import List
import warnings

warnings.filterwarnings("ignore")

Before defining our inputs, we first define the power law schedule function:

In [6]:
def power_law_schedule(eps_precision: float = 0.01,
                       beta: float = 0.455) -> List[int]:
    """
    Power Law schedule

    Parameters
    ----------
    eps_precision : float, optional
        Epsilon precision. The default is 0.01.
    beta : float, optional
        Beta. The default is 0.455.

    Returns
    -------
    List[int]
        Power Law schedule.

    """
    max_k = max(int(1/eps_precision ** (2*beta)), int(math.log(1/eps_precision)))
    evaluation_schedule = [int(k ** ((1-beta)/(2*beta))) for k in range(1, max_k+1)]
    return evaluation_schedule

We can now introduce the user inputs and print the configuration:

In [9]:
##############
# (1) Inputs #
##############
""" Start Inputs """
# Number of qubits the operator A acts on ("n" in the Suzuki paper).
# Notice that the measurement qubit is not included here.
# By using more qubits, we can discretized (approximate) the integral better
n_qubits_input = 3
qubits_per_qpu = [10, 10, 10]
problem = 'sine_squared'  # 'sine_squared' for computing the integral of (sin(x))^2
evaluation_schedule = power_law_schedule()  # power_law_schedule() for power law, None for Suzuki
""" End Inputs """

# Make config for the program
config = QAEMonteCarloConfig(n_qubits_input, qubits_per_qpu, problem,
                             evaluation_schedule=evaluation_schedule)
config.print_configuration()

### Configuration ###
# Algorithm Configuration #
N qubits: 3, N circuits: 66
# Hardware Configuration #
N QPUs: 3, qubits per qpu: [10, 10, 10]
# Integral Configuration #
Problem: sine_squared, b_max: 0.6283185307179586



We define a `make_operator` function which returns the wanted operator, given the configuration:

In [10]:
def make_operator(integral_config: IntegralConfig, n_qubits: int) -> UnitaryOperator:
    """
    Make problem operator. Only SineSquaredOperator supported so far.

    Parameters
    ----------
    integral_config : IntegralConfig
    n_qubits : int

    Raises
    ------
    ValueError
        If integral_config.problem is not sine_squared.

    Returns
    -------
    Type[UnitaryOperator]
        Problem operator.

    """
    if integral_config.problem == 'sine_squared':
        return SineSquaredOperator(n_qubits, integral_config.param)
    else:
        raise ValueError("Operator for problem {} not implemented".format(
            integral_config.problem
        ))

Let's build all the operators making up the circuits:

In [11]:
#######################
# (2) Operators A & Q #
#######################
# Make the A and Q operators specific to the problem to estimate.
# Currently only SineSquaredOperator is implemented.
problem_operator = make_operator(config.integral, config.algo.n_qubits)
state_preparation = problem_operator.prepare_state()  # P x R operator
grover_operator = problem_operator.make_grover()  # Q operator

We now define the estimation problem, using the Qiskit built-in `EstimationProblem` class:

In [12]:
#####################################
# (3) Set up the estimation problem #
#####################################
problem = EstimationProblem(
    state_preparation=state_preparation,
    objective_qubits=problem_operator.oracle_size - 1,  # -1 as it's the index of the qubit to measure
    grover_operator=grover_operator
)

In order to run the estimation problem parallely, we need to define a distribution schedule (in our case a greedy distribution schedule):

In [13]:
##########################################
# (4) Schedule for distributed computing #
##########################################
# A greedy schedule is a schedule that greedily fills the QPUs with as many qubits
# that can possibly fit; when the QPUs cannot fit any more qubits, the execution
# of those estimations are moved to the next round.
# Other approaches are possible, even though not implemented, such as constraint programming.
greedy_schedule = GreedySchedule(
    config.algo.num_circuits, config.hardware, problem_operator.oracle_size,
    allow_distributed=False
)
greedy_schedule.make_schedule()
if len(greedy_schedule.schedule) == 0:
    raise SystemExit("Hardware capabilities are too small to handle the size of the oracle.")
else:
    greedy_schedule.print_schedule()
# greedy_schedule.schedule is a dictionary having:
# - rounds as keys
# - list of tuples (circuit_id, [qubits_qpu1, .., qubits_qpuN])

### Schedule for parallelization ###
# Round 1 #
[(0, [5, 0, 0]), (1, [0, 5, 0]), (2, [0, 0, 5]), (3, [0, 0, 5]), (4, [0, 5, 0]), (5, [5, 0, 0])]
# Round 2 #
[(0, [5, 0, 0]), (1, [0, 5, 0]), (2, [0, 0, 5]), (3, [0, 0, 5]), (4, [0, 5, 0]), (5, [5, 0, 0])]
# Round 3 #
[(0, [5, 0, 0]), (1, [0, 5, 0]), (2, [0, 0, 5]), (3, [0, 0, 5]), (4, [0, 5, 0]), (5, [5, 0, 0])]
# Round 4 #
[(0, [5, 0, 0]), (1, [0, 5, 0]), (2, [0, 0, 5]), (3, [0, 0, 5]), (4, [0, 5, 0]), (5, [5, 0, 0])]
# Round 5 #
[(0, [5, 0, 0]), (1, [0, 5, 0]), (2, [0, 0, 5]), (3, [0, 0, 5]), (4, [0, 5, 0]), (5, [5, 0, 0])]
# Round 6 #
[(0, [5, 0, 0]), (1, [0, 5, 0]), (2, [0, 0, 5]), (3, [0, 0, 5]), (4, [0, 5, 0]), (5, [5, 0, 0])]
# Round 7 #
[(0, [5, 0, 0]), (1, [0, 5, 0]), (2, [0, 0, 5]), (3, [0, 0, 5]), (4, [0, 5, 0]), (5, [5, 0, 0])]
# Round 8 #
[(0, [5, 0, 0]), (1, [0, 5, 0]), (2, [0, 0, 5]), (3, [0, 0, 5]), (4, [0, 5, 0]), (5, [5, 0, 0])]
# Round 9 #
[(0, [5, 0, 0]), (1, [0, 5, 0]), (2, [0, 0, 5]), (3, [0, 0, 5]), (4, [0, 5, 0])

We are now ready to set up the MLAE algorithm:

In [14]:
##########################################
# (5) Set up the parallel MLAE algorithm #
##########################################
# PMLAE needs, as well as an evaluation schedule, a parallelization schedule
# and a ParallelQuantumInstance
pmlae = ParallelMaximumLikelihoodAmplitudeEstimation(
    evaluation_schedule=config.algo.evaluation_schedule,
    parallelization_schedule=greedy_schedule,
    quantum_instance=ParallelQuantumInstance(Aer.get_backend('qasm_simulator'))
    )

We can now run the algorithm given the estimation problem:

In [15]:
##################################
# (6) Run the estimation problem #
##################################
result = pmlae.estimate(problem)

And finally print the results:

In [16]:
#####################
# (7) Print results #
#####################
print("### Results ###")
print("Analytical result: {}".format(problem_operator.analytical_result()))
print("Discretized result: {}".format(problem_operator.discretized_result()))
print("Estimation result: {}".format(result.estimation))

### Results ###
Analytical result: 0.12158663567967151
Discretized result: 0.1211973148745352
Estimation result: 0.12088021456797103
